# Exploratory Data Analysis (EDA) - Employee Analysis

This notebook performs comprehensive exploratory data analysis on employee datasets to understand employee differences and patterns.


## 1. Import Required Libraries

In [ ]:
# Data manipulation and analysis
import os
import sys
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns

# Statistical analysis
from scipy.stats import  shapiro


# Utils functions
src_path = os.path.abspath("../../src")
if src_path not in sys.path:
    sys.path.append(src_path)

from utils import analyze_dataset, detect_outliers_iqr, detect_outliers_zscore  # type: ignore

# Set visualization style
plt.style.use('default')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 2. Load Individual Datasets

In [ ]:
# Load the three starting datasets
df_sirh = pd.read_csv('../../data/raw/extrait_sirh.csv')  # SIRH dataset
df_eval = pd.read_csv('../../data/raw/extrait_eval.csv')  # Evaluation dataset
df_sondage = pd.read_csv('../../data/raw/extrait_sondage.csv')  # Survey dataset

dfs = {
    "sirh": df_sirh,
    "eval": df_eval,
    "sondage": df_sondage,
}


## 3. Descriptive Statistics on Individual Files

Analyze each file separately before joining to understand their individual characteristics.

In [ ]:
# Analyze each dataset

for name, df in dfs.items():
	analyze_dataset(df, name)



## 4. Data Visualization

Visualize individual datasets to understand distributions and patterns before merging.

In [ ]:
print("=== VISUALISATIONS: Histogram + Boxplot + QQ-plot ===")

for name, df in dfs.items():
    numerical_columns = df.select_dtypes(include=[np.number]).columns.tolist()

    # Filtrer les colonnes ID
    id_columns = [
        col
        for col in numerical_columns
        if "id" in col.lower() or "number" in col.lower() or "code" in col.lower()
    ]
    meaningful_cols = [col for col in numerical_columns if col not in id_columns]

    if len(meaningful_cols) > 0:
        print(f"\n📊 {name} - variables numériques: {meaningful_cols}")

        for col in meaningful_cols:
            data = df[col].dropna()
            if len(data) < 5:
                continue  # trop peu de valeurs

            fig, axes = plt.subplots(1, 3, figsize=(18, 5))

            # Histogramme + KDE
            sns.histplot(
                data,
                bins=20,
                kde=True,
                ax=axes[0],
                color="lightblue",
                edgecolor="black",
            )
            axes[0].set_title(f"{col} - Histogram + KDE")
            axes[0].set_xlabel("Valeurs")
            axes[0].set_ylabel("Fréquence")

            # Boxplot
            sns.boxplot(x=data, ax=axes[1], color="lightblue")
            axes[1].set_title(f"{col} - Boxplot")
            axes[1].set_xlabel("Valeurs")

            # QQ-plot (normalité)
            stats.probplot(data, dist="norm", plot=axes[2])
            axes[2].set_title(f"{col} - QQ-Plot")

            plt.suptitle(f"{name} - {col}", fontsize=14, fontweight="bold")
            plt.tight_layout()
            plt.show()

    else:
        print(f"❌ {name}: uniquement des colonnes ID/techniques → rien de tracé.")

In [ ]:
# Bar plots for categorical variables in individual datasets
print("=== BAR PLOTS FOR INDIVIDUAL DATASETS ===")

for name, df in dfs.items():
    categorical_columns = df.select_dtypes(include=['object']).columns
    
    if len(categorical_columns) > 0:
        print(f"\nCreating bar plots for {name}...")
        
        # Calculate subplot dimensions
        n_cols = min(2, len(categorical_columns))
        n_rows = (len(categorical_columns) + 1) // 2
        
        fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(15, 5*n_rows))
        
        # Handle single subplot case
        if len(categorical_columns) == 1:
            axes = [axes]
        elif n_rows == 1:
            axes = [axes] if n_cols == 1 else axes
        else:
            axes = axes.ravel()
        
        for i, col in enumerate(categorical_columns):
            if i < len(axes):
                value_counts = df[col].value_counts().head(10)  # Top 10 values
                value_counts.plot(kind='bar', ax=axes[i])
                axes[i].set_title(f'{name} - Distribution of {col}')
                axes[i].set_xlabel(col)
                axes[i].set_ylabel('Count')
                axes[i].tick_params(axis='x', rotation=45)
        
        # Hide empty subplots
        for i in range(len(categorical_columns), len(axes)):
            axes[i].set_visible(False)
        
        plt.suptitle(f'{name} - Categorical Variables Distribution')
        plt.tight_layout()
        plt.show()
    else:
        print(f"{name}: No categorical variables found for bar plots.")

## 5. Distribution Analysis

In [ ]:
# Normality tests for numerical variables in each dataset
print("=== NORMALITY TESTS FOR INDIVIDUAL DATASETS ===")
print("Shapiro-Wilk test:")
print("H0: Data is normally distributed")
print("H1: Data is not normally distributed")
print("If p-value < 0.05, reject H0 (data is not normally distributed)\n")

for name, df in dfs.items():
    numerical_columns = df.select_dtypes(include=[np.number]).columns
    
    if len(numerical_columns) > 0:
        print(f"\n{name} - Normality Tests:")
        print("-" * 40)
        
        for col in numerical_columns:
            if len(df[col].dropna()) > 3:  # Need at least 3 values for the test
                statistic, p_value = shapiro(df[col].dropna())
                result = "Normal" if p_value > 0.05 else "Not Normal"
                print(f"{col}: p-value = {p_value:.6f} -> {result}")
        
        print(f"\n{name} - Skewness and Kurtosis:")
        for col in numerical_columns:
            skew = df[col].skew()
            kurt = df[col].kurtosis()
            print(f"{col}: Skewness = {skew:.3f}, Kurtosis = {kurt:.3f}")
    else:
        print(f"{name}: No numerical variables found for normality tests.")
    
    print("\n" + "="*60)

## 6. Outlier Detection

In [ ]:
# Outlier detection using IQR method for each dataset

print("=== OUTLIER DETECTION FOR INDIVIDUAL DATASETS ===")
print("Using IQR method:")
print("="*60)

for name, df in dfs.items():
    numerical_columns = df.select_dtypes(include=[np.number]).columns
    
    if len(numerical_columns) > 0:
        print(f"\n{name} - Outlier Analysis:")
        print("-" * 40)
        
        for col in numerical_columns:
            outliers, lower_bound, upper_bound = detect_outliers_iqr(df, col)
            outlier_count = len(outliers)
            outlier_percentage = (outlier_count / len(df)) * 100
            
            print(f"\n{col}:")
            print(f"  - Outliers count: {outlier_count} ({outlier_percentage:.2f}%)")
            print(f"  - Lower bound: {lower_bound:.3f}")
            print(f"  - Upper bound: {upper_bound:.3f}")
            
            if outlier_count > 0 and outlier_count <= 10:
                print(f"  - Outlier values: {outliers[col].tolist()}")
    else:
        print(f"{name}: No numerical variables found for outlier detection.")
    
    print("\n" + "="*60)

In [ ]:
print("=== OUTLIER DETECTION FOR INDIVIDUAL DATASETS ===")
print("Using Z-score method:")
print("=" * 60)

for name, df in dfs.items():
    numerical_columns = df.select_dtypes(include=[np.number]).columns

    if len(numerical_columns) > 0:
        print(f"\n{name} - Outlier Analysis:")
        print("-" * 40)

        for col in numerical_columns:
            outliers, z_scores = detect_outliers_zscore(df, col, threshold=3.0)
            outlier_count = len(outliers)
            outlier_percentage = (outlier_count / len(df)) * 100

            print(f"\n{col}:")
            print(f"  - Outliers count: {outlier_count} ({outlier_percentage:.2f}%)")
            print(f"  - Threshold: |Z| > 3.0")

            if outlier_count > 0 and outlier_count <= 10:
                print(f"  - Outlier values: {outliers[col].tolist()}")
    else:
        print(f"{name}: No numerical variables found for outlier detection.")

    print("\n" + "=" * 60)

In [ ]:
df_eval["niveau_hierarchique_poste"].describe()

## 7. Data cleaning

In [ ]:
df_sirh["statut_marital"].unique()

In [ ]:
# Delete nombre_heures_travailless - same value for all rows
if len(df_sirh['nombre_heures_travailless'].value_counts()) == 1:
    df_sirh.drop(columns=['nombre_heures_travailless'], inplace=True)
    
df_sirh.columns

In [ ]:
# Delete nombre_employee_sous_responsabilite - same value for all rows
if len(df_sondage['nombre_employee_sous_responsabilite'].value_counts()) == 1:
	df_sondage.drop(columns=['nombre_employee_sous_responsabilite'], inplace=True)

df_sondage.columns

In [ ]:
# Delete ayant_enfants - same value for all rows
if len(df_sondage['ayant_enfants'].value_counts()) == 1:
    df_sondage.drop(columns=['ayant_enfants'], inplace=True)
    
df_sondage.columns

## 8. Create Central DataFrame by Joining

After completing individual dataset analysis, create the central DataFrame by joining all datasets.
Join type used is `left` to retain all employees from SIRH dataset.

In [ ]:
# Merge both df_sirh on 'id_employee' (ex: 10) and df_eval datasets on 'eval_number' ex("E_10")
# Clean eval_number column if it contains 'E_' prefix
if df_eval['eval_number'].dtype == 'object':
    df_eval['eval_number'] = df_eval['eval_number'].str.replace('E_', '').astype(int)

# Ensure id_employee is integer
df_sirh['id_employee'] = df_sirh['id_employee'].astype(int)

# Perform the merges
df_sirh_eval = pd.merge(df_sirh, df_eval, left_on='id_employee', right_on='eval_number', how='left')
df_sirh_eval_sondage = pd.merge(df_sirh_eval, df_sondage, left_on='id_employee', right_on='code_sondage', how='left')

analyze_dataset(df_sirh_eval_sondage, "FINAL MERGED DATASET (SIRH, EVALUATION & SURVEY)")

## 9. Visualizations to Highlight Employee Differences

**Choose appropriate chart types**:
- **Quanti vs Quanti**: Scatter plots, correlation heatmaps
- **Quanti vs Quali**: Box plots, violin plots, grouped bar charts
- **Quali vs Quali**: Stacked bar charts, crosstab heatmaps

In [ ]:
df_sirh_eval_sondage.columns

In [ ]:
# Split the final merged dataset into two: those who left and those who stayed
df_left = df_sirh_eval_sondage[df_sirh_eval_sondage['a_quitte_l_entreprise'] == 1]
df_stayed = df_sirh_eval_sondage[df_sirh_eval_sondage['a_quitte_l_entreprise'] == 0]

In [ ]:
group_means = df_sirh_eval_sondage.groupby("a_quitte_l_entreprise")[['revenu_mensuel',
       'nombre_experiences_precedentes', 'annee_experience_totale',
       'annees_dans_l_entreprise', 'annees_dans_le_poste_actuel',
       'satisfaction_employee_environnement', 'note_evaluation_precedente',
       'niveau_hierarchique_poste', 'satisfaction_employee_nature_travail',
       'satisfaction_employee_equipe',
       'satisfaction_employee_equilibre_pro_perso', 
       'note_evaluation_actuelle', 'nombre_participation_pee',
       'nb_formations_suivies', 'distance_domicile_travail',
       'niveau_education', 'annees_depuis_la_derniere_promotion',
       'annes_sous_responsable_actuel']].mean()
group_labels = group_means.index.tolist()
diff_means = group_means.loc[group_labels[1]] - group_means.loc[group_labels[0]]


print("Moyennes par groupe :\n", group_means)
print("\nDifférence de moyenne :\n", diff_means)

In [ ]:
numerical_cols = df_sirh_eval_sondage.select_dtypes(include="number").columns.tolist()
# Exclure toutes les colonnes ID ou non-significatives
exclude_cols = ["id_employee", "eval_number", "code_sondage"]
numerical_cols = [c for c in numerical_cols if c not in exclude_cols]

for col in numerical_cols:
    plt.figure(figsize=(6, 4))
    sns.histplot(
        data=df_sirh_eval_sondage,
        x=col,
        hue="a_quitte_l_entreprise",
        stat="density",
        common_norm=False,
        kde=True,
        palette=["skyblue", "salmon"],
        alpha=0.6,
    )
    plt.title(f"Distribution of {col} by turnover")
    plt.show()

In [ ]:
sns.histplot(
	df_sirh_eval_sondage, x="age", hue="a_quitte_l_entreprise", kde=True
)

In [ ]:
sns.countplot(x="departement", hue="a_quitte_l_entreprise", data=df_sirh_eval_sondage)

In [ ]:
sns.boxplot(x="a_quitte_l_entreprise", y="annees_dans_l_entreprise", data=df_sirh_eval_sondage)

## Optional : Merge Verification and Validation

Let's verify the merge operations step by step to ensure data integrity.

In [ ]:
print("=== MERGE VERIFICATION ANALYSIS ===")
print("="*60)

# 1. Original dataset sizes
print("1. ORIGINAL DATASET SIZES:")
print(f"   SIRH dataset: {df_sirh.shape[0]} rows, {df_sirh.shape[1]} columns")
print(f"   Evaluation dataset: {df_eval.shape[0]} rows, {df_eval.shape[1]} columns") 
print(f"   Survey dataset: {df_sondage.shape[0]} rows, {df_sondage.shape[1]} columns")

# 2. Check uniqueness of join keys (current state)
print("\n2. JOIN KEY UNIQUENESS (CURRENT STATE):")
print(f"   SIRH 'id_employee' unique values: {df_sirh['id_employee'].nunique()} / {len(df_sirh)} ({df_sirh['id_employee'].nunique()/len(df_sirh)*100:.1f}% unique)")
print(f"   Evaluation 'eval_number' unique values: {df_eval['eval_number'].nunique()} / {len(df_eval)} ({df_eval['eval_number'].nunique()/len(df_eval)*100:.1f}% unique)")
print(f"   Survey 'code_sondage' unique values: {df_sondage['code_sondage'].nunique()} / {len(df_sondage)} ({df_sondage['code_sondage'].nunique()/len(df_sondage)*100:.1f}% unique)")

# 3. Check the data types and sample values
print("\n3. KEY DATA TYPES AND SAMPLES:")
print(f"   SIRH id_employee type: {df_sirh['id_employee'].dtype}, samples: {df_sirh['id_employee'].head(3).tolist()}")
print(f"   Evaluation eval_number type: {df_eval['eval_number'].dtype}, samples: {df_eval['eval_number'].head(3).tolist()}")
print(f"   Survey code_sondage type: {df_sondage['code_sondage'].dtype}, samples: {df_sondage['code_sondage'].head(3).tolist()}")

# 4. Check overlaps between keys
print("\n4. KEY OVERLAP ANALYSIS:")
sirh_ids = set(df_sirh['id_employee'])
eval_ids = set(df_eval['eval_number'])
survey_ids = set(df_sondage['code_sondage'])

overlap_sirh_eval = sirh_ids & eval_ids
overlap_sirh_survey = sirh_ids & survey_ids
overlap_eval_survey = eval_ids & survey_ids
all_overlap = sirh_ids & eval_ids & survey_ids

print(f"   SIRH ∩ Evaluation: {len(overlap_sirh_eval)} common IDs")
print(f"   SIRH ∩ Survey: {len(overlap_sirh_survey)} common IDs") 
print(f"   Evaluation ∩ Survey: {len(overlap_eval_survey)} common IDs")
print(f"   All three datasets: {len(all_overlap)} common IDs")

# 5. Merged dataset sizes
print("\n5. MERGED DATASET SIZES:")
print(f"   SIRH + Evaluation: {df_sirh_eval.shape[0]} rows, {df_sirh_eval.shape[1]} columns")
print(f"   Final (all three): {df_sirh_eval_sondage.shape[0]} rows, {df_sirh_eval_sondage.shape[1]} columns")

print("\n" + "="*60)

In [ ]:
print("=== DETAILED MERGE VALIDATION ===")
print("="*50)

# 6. Check for data loss/duplication in merges
print("6. MERGE IMPACT ANALYSIS:")

# First merge (SIRH + Evaluation)
sirh_lost = len(df_sirh) - len(df_sirh_eval[df_sirh_eval['eval_number'].notna()])
eval_lost = len(df_eval) - len(df_sirh_eval[df_sirh_eval['eval_number'].notna()])

print(f"   First merge (SIRH + Evaluation with LEFT join):")
print(f"     - SIRH records retained: {len(df_sirh_eval)} / {len(df_sirh)} ({len(df_sirh_eval)/len(df_sirh)*100:.1f}%)")
print(f"     - Evaluation data matched: {len(df_sirh_eval[df_sirh_eval['eval_number'].notna()])} / {len(df_eval)} ({len(df_sirh_eval[df_sirh_eval['eval_number'].notna()])/len(df_eval)*100:.1f}%)")

# Second merge (+ Survey)
sirh_eval_lost = len(df_sirh_eval) - len(df_sirh_eval_sondage[df_sirh_eval_sondage['code_sondage'].notna()])
survey_lost = len(df_sondage) - len(df_sirh_eval_sondage[df_sirh_eval_sondage['code_sondage'].notna()])

print(f"\n   Second merge (+ Survey with LEFT join):")
print(f"     - SIRH+Eval records retained: {len(df_sirh_eval_sondage)} / {len(df_sirh_eval)} ({len(df_sirh_eval_sondage)/len(df_sirh_eval)*100:.1f}%)")
print(f"     - Survey data matched: {len(df_sirh_eval_sondage[df_sirh_eval_sondage['code_sondage'].notna()])} / {len(df_sondage)} ({len(df_sirh_eval_sondage[df_sirh_eval_sondage['code_sondage'].notna()])/len(df_sondage)*100:.1f}%)")

# 7. Check for duplicates after merge
print("\n7. DUPLICATE CHECK AFTER MERGES:")
print(f"   Duplicates in SIRH+Eval: {df_sirh_eval.duplicated().sum()}")
print(f"   Duplicates in final dataset: {df_sirh_eval_sondage.duplicated().sum()}")

# 8. Missing values introduced by merges
print("\n8. MISSING VALUES INTRODUCED:")
sirh_eval_missing = df_sirh_eval.isnull().sum().sum() - df_sirh.isnull().sum().sum()
final_missing = df_sirh_eval_sondage.isnull().sum().sum() - df_sirh_eval.isnull().sum().sum()

print(f"   Missing values added in first merge: {sirh_eval_missing}")
print(f"   Missing values added in second merge: {final_missing}")

print("\n" + "="*50)

In [ ]:
print("=== MERGE QUALITY ASSESSMENT ===")
print("="*45)

# 9. Sample verification - show some merged records
print("9. SAMPLE MERGED DATA VERIFICATION:")
print("\nFirst 3 records with all data present:")
complete_records = df_sirh_eval_sondage[
    (df_sirh_eval_sondage['eval_number'].notna()) & 
    (df_sirh_eval_sondage['code_sondage'].notna())
]

if len(complete_records) > 0:
    sample_cols = ['id_employee', 'eval_number', 'code_sondage']
    available_cols = [col for col in sample_cols if col in complete_records.columns]
    print(complete_records[available_cols].head(3))
    print(f"\nTotal records with all three sources: {len(complete_records)}")
else:
    print("No records found with data from all three sources!")

# 10. Key relationship verification
print("\n10. KEY RELATIONSHIP VERIFICATION:")
print("Checking if id_employee = eval_number = code_sondage after formatting...")

if len(complete_records) > 0:
    # Check if the keys match for records that have all data
    id_eval_match = (complete_records['id_employee'] == complete_records['eval_number']).all()
    id_survey_match = (complete_records['id_employee'] == complete_records['code_sondage']).all()
    
    print(f"   id_employee == eval_number: {id_eval_match}")
    print(f"   id_employee == code_sondage: {id_survey_match}")
    
    if id_eval_match and id_survey_match:
        print("   ✅ All keys match perfectly - merge is correct!")
    else:
        print("   ⚠️  Key mismatch detected - needs investigation")
        
        # Show mismatches
        id_eval_mismatch = complete_records[complete_records['id_employee'] != complete_records['eval_number']]
        id_survey_mismatch = complete_records[complete_records['id_employee'] != complete_records['code_sondage']]
        
        if len(id_eval_mismatch) > 0:
            print(f"   ID-Eval mismatches: {len(id_eval_mismatch)} records")
        if len(id_survey_mismatch) > 0:
            print(f"   ID-Survey mismatches: {len(id_survey_mismatch)} records")

print("\n" + "="*45)

In [ ]:
# Save the cleaned and merged dataset
print("💾 Saving cleaned dataset...")

import os

# Create the processed directory if it doesn't exist
processed_dir = '../../data/processed'
os.makedirs(processed_dir, exist_ok=True)

# Remove redundant id columns after merge
df_sirh_eval_sondage.drop(columns=["eval_number", "code_sondage"], inplace=True)

# Save the final merged dataset (overwrite existing file)
output_path = '../../data/processed/employee_data_merged_clean.csv'
df_sirh_eval_sondage.to_csv(output_path, index=False)

print(f"✅ Dataset saved to: {output_path}")
print(f"📊 Dataset contains: {df_sirh_eval_sondage.shape[0]} employees with {df_sirh_eval_sondage.shape[1]} features")
print("📝 File will be overwritten on each run - no timestamped backups")